# এজেন্টিক RAG Autogen-এর সাথে

এই নোটবুকটি উন্নত মূল্যায়ন ক্ষমতা সহ Autogen এজেন্ট ব্যবহার করে Retrieval-Augmented Generation (RAG) বাস্তবায়নের প্রদর্শন করে।


SQLite সংস্করণ সংশোধন  
যদি আপনি এই ত্রুটির সম্মুখীন হন:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

আপনার নোটবুকের শুরুতে এই কোড ব্লকটি আনকমেন্ট করুন:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## ক্লায়েন্ট তৈরি করুন

প্রথমে, আমরা Azure AI Chat Completion Client আরম্ভ করি। এই ক্লায়েন্টটি ব্যবহারকারীর প্রশ্নের উত্তর তৈরি করতে Azure OpenAI পরিষেবার সাথে যোগাযোগ করার জন্য ব্যবহৃত হবে।


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## ভেক্টর ডাটাবেস ইনিশিয়ালাইজেশন

আমরা স্থায়ী স্টোরেজ সহ ChromaDB ইনিশিয়ালাইজ করি এবং উন্নত নমুনা ডকুমেন্ট যোগ করি। ChromaDB ব্যবহার করা হবে ডকুমেন্ট সংরক্ষণ এবং পুনরুদ্ধারের জন্য, যা সঠিক উত্তর তৈরির প্রেক্ষাপট প্রদান করবে।


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## কনটেক্সট প্রদানকারী বাস্তবায়ন

`ContextProvider` ক্লাসটি বিভিন্ন উৎস থেকে কনটেক্সট সংগ্রহ এবং সংহত করার কাজ করে:

1. **ভেক্টর ডাটাবেস সংগ্রহ**: ChromaDB ব্যবহার করে ভ্রমণ সংক্রান্ত ডকুমেন্টগুলোর উপর সেমান্টিক সার্চ পরিচালনা করে
2. **আবহাওয়ার তথ্য**: প্রধান শহরগুলোর জন্য একটি সিমুলেটেড আবহাওয়া ডাটাবেস বজায় রাখে
3. **একীভূত কনটেক্সট**: ডকুমেন্ট এবং আবহাওয়ার তথ্য একত্রিত করে একটি পূর্ণাঙ্গ কনটেক্সট তৈরি করে

মূল পদ্ধতিগুলো:
- `get_retrieval_context()`: প্রশ্নের ভিত্তিতে প্রাসঙ্গিক ডকুমেন্ট সংগ্রহ করে
- `get_weather_data()`: নির্দিষ্ট স্থানের জন্য আবহাওয়ার তথ্য প্রদান করে
- `get_unified_context()`: ডকুমেন্ট এবং আবহাওয়ার কনটেক্সট একত্রিত করে উন্নত প্রতিক্রিয়া প্রদান করে


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## এজেন্ট কনফিগারেশন

আমরা রিট্রিভাল এবং অ্যাসিস্ট্যান্ট এজেন্ট কনফিগার করি। রিট্রিভাল এজেন্ট সেমান্টিক সার্চ ব্যবহার করে প্রাসঙ্গিক তথ্য খুঁজে বের করতে বিশেষজ্ঞ, আর অ্যাসিস্ট্যান্ট পুনরুদ্ধার করা তথ্যের ভিত্তিতে বিস্তারিত উত্তর তৈরি করে।


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG দিয়ে প্রশ্ন প্রক্রিয়াকরণ

আমরা `ask_rag` ফাংশনটি সংজ্ঞায়িত করি, যা প্রশ্নটি সহকারীকে পাঠায়, উত্তরটি প্রক্রিয়া করে এবং তা মূল্যায়ন করে। এই ফাংশনটি সহকারীর সাথে যোগাযোগ পরিচালনা করে এবং উত্তরটির গুণমান পরিমাপ করতে মূল্যায়ক ব্যবহার করে।


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# উদাহরণ ব্যবহার

আমরা evaluator আরম্ভ করি এবং সেই প্রশ্নগুলো নির্ধারণ করি যেগুলো আমরা প্রক্রিয়া করতে এবং মূল্যায়ন করতে চাই।


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## স্ক্রিপ্ট চালান

আমরা যাচাই করি স্ক্রিপ্টটি ইন্টারঅ্যাকটিভ পরিবেশে চলছে কিনা বা একটি স্ট্যান্ডার্ড স্ক্রিপ্টে, এবং সেই অনুযায়ী প্রধান ফাংশনটি চালাই।


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
